In [25]:
import os
import h5py

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from astropy.cosmology import Planck18  # Planck 2018

import agama
import gizmo_analysis as ga
import utilities as ut

%matplotlib inline
plt.style.use('../matplotlib_style/latex_plot_style.mplstyle')

In [2]:
# Load in potential with agama
gal = 'm12i_res7100'
DM_pot_file = f'potentials/{gal}/600.dark.axi_4.coef_mul_DR'   # DM halo + cold gas (multiple)
bar_pot_file = f'potentials/{gal}/600.bar.axi_4.coef_cylsp_DR' # bar + hot gas (cubicspline)
pxr_DM = agama.Potential(file=DM_pot_file)
pxr_bar = agama.Potential(file=bar_pot_file)

print('Files loaded!')
pot_model = agama.Potential(pxr_DM, pxr_bar)

Files loaded!


In [31]:
# Read in the id and redshift of accreted stars
table = pd.read_csv(f'accretion_history/stars_accretion_history_{gal}_v2.csv')
idx = table['id_stars'].values
zacc = table['redshift_stars'] .values

# calculate the lookback time (in Gyr) for all redshifts
time = -Planck18.lookback_time(zacc).value

In [6]:
# Picking a sample simulation and a snapshot
sim_dir = f'/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/{gal}'
nsnap = 600

# Read using gizmo
part = ga.io.Read.read_snapshots(species=['star'],
                                snapshot_values=nsnap,
                                simulation_directory=sim_dir,
                                assign_hosts=True,
                                assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100/snapshot_times.txt

  using snapshot index = 600, redshift = 0.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 57060074 particles
    star      (id = 4): 13976485 particles
    blackhole (id = 5): 0 particles

* reading species: ['star']
* reading particles from:
    snapshot_600.0.hdf5
    snapshot_600.1.hdf5
    snapshot_600.2.hdf5
    snapshot_600.3.hdf5

* reading cosmological parameters from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoo

In [47]:
# Get the position and velocit of all accreted stars
pos = part['star'].prop("host.distance.principal")
vel = part['star'].prop("host.velocity.principal")
pos = pos[idx]
vel = vel[idx]
pos_vel = np.hstack([pos, vel])

In [ ]:
peris = []
apos = []
for i in range(len(orbit)):
    r = np.linalg.norm(orbit[0][1][:, :3], axis=1)
    peris.append(np.min(r))
    apos.append(np.max(r))
peris = np.array(peris)
apos = np.array(apos)

In [111]:
# Integrate orbit (at least to 7 Gyr)
orbit = agama.orbit(ic=pos_vel, potential=pot_model, time=np.minimum(time, -7), trajsize=100)

10 orbits complete (1.111 orbits/s)
3118/173333 or


KeyboardInterrupt



66